# Load dataset
Import Porto dataset from UP google drive.

(You should make a private copy of the dataset before running this script)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls '/content/gdrive/My Drive'

Create a duplicate of the dataset

And create the train and test folder

In [0]:
!cp -r '/content/gdrive/My Drive/porto-dataset' '/content/gdrive/My Drive/porto-dataset-2'
!mkdir -p /content/gdrive/My\ Drive/porto-dataset-2/images/test
!mkdir -p /content/gdrive/My\ Drive/porto-dataset-2/images/train

## Move images
Move the first 20% of each category to test

In [0]:
%cd /content/gdrive/My\ Drive/porto-dataset-2/images/
%cd arrabida
!mv `ls | head -120` ../test/
%cd ../camara
!mv `ls | head -120` ../test/
%cd ../clerigos
!mv `ls | head -120` ../test/
%cd ../musica
!mv `ls | head -120` ../test/
%cd ../serralves
!mv `ls | head -120` ../test/
%cd /content/gdrive/My\ Drive/porto-dataset-2/

Move all the other images to train

In [0]:
!mv images/arrabida/*.* images/train
!mv images/camara/*.* images/train
!mv images/clerigos/*.* images/train
!mv images/musica/*.* images/train
!mv images/serralves/*.* images/train

## Move annotations
Move the first 20% to test

In [0]:
%cd /content/gdrive/My\ Drive/porto-dataset-2/annotations/
%cd arrabida
!mv `ls | head -120` /content/gdrive/My\ Drive/porto-dataset-2/images/test
%cd ../camara
!mv `ls | head -120` /content/gdrive/My\ Drive/porto-dataset-2/images/test
%cd ../clerigos
!mv `ls | head -120` /content/gdrive/My\ Drive/porto-dataset-2/images/test
%cd ../musica
!mv `ls | head -120` /content/gdrive/My\ Drive/porto-dataset-2/images/test
%cd ../serralves
!mv `ls | head -120` /content/gdrive/My\ Drive/porto-dataset-2/images/test
%cd /content/gdrive/My\ Drive/porto-dataset-2

Move the other annotations to train

In [0]:
!mv annotations/arrabida/*.* images/train
!mv annotations/camara/*.* images/train
!mv annotations/clerigos/*.* images/train
!mv annotations/musica/*.* images/train
!mv annotations/serralves/*.* images/train

Remove unused directories

In [0]:
%cd /content/gdrive/My\ Drive/porto-dataset-2/
!rm -r annotations
!rmdir images/arrabida
!rmdir images/camara
!rmdir images/clerigos
!rmdir images/musica
!rmdir images/serralves

## Convert XML to CSV
The algorithm that is going to be used needs CSV files instead of XML.

So, first we need to convert our bounding box files to CSV.

In [0]:
%cd /content/gdrive/My\ Drive/porto-dataset-2/

!curl -LO https://github.com/MiguelLucas/Porto-Landmarks/blob/master/xml_to_csv.py
  
!python xml_to_csv.py

## Generate tensorflow records
The TFRecord file format is a simple record-oriented binary format that many TensorFlow applications use for training data.

In [0]:
%cd /content/gdrive/My\ Drive/porto-dataset-2/

!curl -LO https://github.com/MiguelLucas/Porto-Landmarks/blob/master/dataset_utils.py
!curl -LO https://github.com/MiguelLucas/Porto-Landmarks/blob/master/generate_tfrecord.py

!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

# Import Tensorflow
Import the code from tensorflow retrain example

https://www.tensorflow.org/hub/tutorials/image_retraining

In [0]:
!curl -LO https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100   158    0     0    346      0 --:--:-- --:--:-- --:--:--   345
100 54688  100 54688    0     0  89213      0 --:--:-- --:--:-- --:--:-- 89213


In [0]:
!python retrain.py --image_dir 'gdrive/My Drive/porto-dataset/images'

INFO:tensorflow:Looking for images in 'arrabida'
INFO:tensorflow:Looking for images in 'camara'
INFO:tensorflow:Looking for images in 'clerigos'
INFO:tensorflow:Looking for images in 'musica'
INFO:tensorflow:Looking for images in 'serralves'
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'.
INFO:tensorflow:Downloaded https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1, Total size: 86.32MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1'.
2018-12-30 13:00:41.077612: W tensorflow/core/graph/graph_constructor.cc:1265] Importing a graph with a lower producer version 26 into an existing graph with producer version 27. Shape inference will have run different parts of the graph with different producer versions.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2018-12-3

# Display Results

In [0]:
!curl -LO https://github.com/tensorflow/tensorflow/raw/master/tensorflow/examples/label_image/label_image.py
!python label_image.py \
  --graph=/tmp/output_graph.pb --labels=/tmp/output_labels.txt \
  --input_layer=Placeholder \
  --output_layer=final_result \
  --image='gdrive/My Drive/porto-dataset/images/arrabida/arrabida-0010.jpg'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   175  100   175    0     0   1508      0 --:--:-- --:--:-- --:--:--  1508
100  4707  100  4707    0     0  24262      0 --:--:-- --:--:-- --:--:--  218k
2018-12-30 14:47:37.536579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-12-30 14:47:37.537028: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-12-30 14:47:37.537070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2018-12-30 14:47:37.817388: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnec

# Display in Tensorboard

In [0]:
!ls 'gdrive/My Drive/porto-dataset'
!git clone https://github.com/mixuala/colab_utils

import os
import colab_utils.tboard

ROOT = %pwd
LOG_DIR = '/tmp/retrain_logs'

colab_utils.tboard.launch_tensorboard(bin_dir=ROOT, log_dir=LOG_DIR)

 annotations			    cfg		      README.gdoc
'Bounding Box Annotations.gsheet'   images	      scripts
'bounding box examples'		    label_map.pbtxt
fatal: destination path 'colab_utils' already exists and is not an empty directory.
ngrok installed
status: tensorboard=True, ngrok=True
tensorboard url= http://6fb15873.ngrok.io


'http://6fb15873.ngrok.io'